# Find Optimal PSF
===========================

- creation 29/06/2016
- author Sylvie Dagoret-Campagne



Find HD158485 with grating

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [2]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [3]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [4]:
from photutils.background import Background2D

In [5]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [6]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-06-30 06:16:56.442242
fits date format for the analysis :  2016-06-30T06:16:56


## Definitions and Constants

In [7]:
object_name='HD158485_grat'

### input files

In [8]:
path='./HD158485_grat'
rootfilename='AssScImHD158485_grat_' 
#NumStart=1
#NumStop=50

#NumStart=51
#NumStop=100

NumStart=1
NumStop=101

### output file (table)

In [9]:
outputtablefile='HD158485_grat_1-101_TablePSF.fits'

### make the filelist

In [10]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [11]:
filelist

['./HD158485_grat/AssScImHD158485_grat_1.fits',
 './HD158485_grat/AssScImHD158485_grat_2.fits',
 './HD158485_grat/AssScImHD158485_grat_3.fits',
 './HD158485_grat/AssScImHD158485_grat_4.fits',
 './HD158485_grat/AssScImHD158485_grat_5.fits',
 './HD158485_grat/AssScImHD158485_grat_6.fits',
 './HD158485_grat/AssScImHD158485_grat_7.fits',
 './HD158485_grat/AssScImHD158485_grat_8.fits',
 './HD158485_grat/AssScImHD158485_grat_9.fits',
 './HD158485_grat/AssScImHD158485_grat_10.fits',
 './HD158485_grat/AssScImHD158485_grat_11.fits',
 './HD158485_grat/AssScImHD158485_grat_12.fits',
 './HD158485_grat/AssScImHD158485_grat_13.fits',
 './HD158485_grat/AssScImHD158485_grat_14.fits',
 './HD158485_grat/AssScImHD158485_grat_15.fits',
 './HD158485_grat/AssScImHD158485_grat_16.fits',
 './HD158485_grat/AssScImHD158485_grat_17.fits',
 './HD158485_grat/AssScImHD158485_grat_18.fits',
 './HD158485_grat/AssScImHD158485_grat_19.fits',
 './HD158485_grat/AssScImHD158485_grat_20.fits',
 './HD158485_grat/AssScImHD15

## Read Input files

In [12]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD158485_grat/AssScImHD158485_grat_1.fits
./HD158485_grat/AssScImHD158485_grat_2.fits
./HD158485_grat/AssScImHD158485_grat_3.fits
./HD158485_grat/AssScImHD158485_grat_4.fits
./HD158485_grat/AssScImHD158485_grat_5.fits
./HD158485_grat/AssScImHD158485_grat_6.fits
./HD158485_grat/AssScImHD158485_grat_7.fits
./HD158485_grat/AssScImHD158485_grat_8.fits
./HD158485_grat/AssScImHD158485_grat_9.fits
./HD158485_grat/AssScImHD158485_grat_10.fits
./HD158485_grat/AssScImHD158485_grat_11.fits
./HD158485_grat/AssScImHD158485_grat_12.fits
./HD158485_grat/AssScImHD158485_grat_13.fits
./HD158485_grat/AssScImHD158485_grat_14.fits
./HD158485_grat/AssScImHD158485_grat_15.fits
./HD158485_grat/AssScImHD158485_grat_16.fits
./HD158485_grat/AssScImHD158485_grat_17.fits
./HD158485_grat/AssScImHD158485_grat_18.fits
./HD158485_grat/AssScImHD158485_grat_19.fits
./HD158485_grat/AssScImHD158485_grat_20.fits
./HD158485_grat/AssScImHD158485_grat_21.fits
./HD158485_grat/AssScImHD158485_grat_22.fits
./HD158485_grat/Ass

In [13]:
#basefile_list

## For control

uncomment for control

In [14]:
index=0

In [15]:
#print exposures_list[index]

In [16]:
#header_list[index]

In [17]:
#plt.imshow(data_list[index])

In [18]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [19]:
#data_list[index].data-bkg.background

## Background subtraction

In [20]:
correctedimage_list = []

In [21]:
for data in data_list:
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=data-bkg.background
    correctedimage_list.append(newimage)

## Calculation of PSF

In [22]:
NBIMAGES=len(correctedimage_list)

In [23]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

In [24]:
image_psf=np.zeros((NBIMAGES,6))

In [25]:
DELTA_NBINSX=10
DELTA_NBINSY=10

In [26]:
# loop on images
index=-1
for image in correctedimage_list:
    index=index+1
    mean, median, std = sigma_clipped_stats(image, sigma=10.0, iters=5) 
    sources = daofind(image - median, fwhm=3.0, threshold=50.*std)
    selected_stars=sources.as_array()
    NBSTARS=selected_stars.shape[0]
    print 'image {} ==> NBSTARS = {}'.format(index,NBSTARS)
    image_psf[index,0]=index
    image_psf[index,1]=NBSTARS
    star_psfx=np.zeros(NBSTARS)
    star_psfy=np.zeros(NBSTARS)
    # loop on stars
    for istar in range(NBSTARS):
        X = int(selected_stars[istar][1])
        Y = int(selected_stars[istar][2])
        prf_image = image[Y-DELTA_NBINSY:Y+DELTA_NBINSY,X-DELTA_NBINSX:X+DELTA_NBINSX]
        oneprfX=prf_image.sum(axis=0)
        oneprfY=prf_image.sum(axis=1)
        if oneprfX.sum() == 0 or oneprfY.sum() == 0:
            star_psfx[istar]=0
            star_psfy[istar]=0
        else:
            posX,sigX=weighted_avg_and_std(np.arange(oneprfX.shape[0]),oneprfX)
            posY,sigY=weighted_avg_and_std(np.arange(oneprfY.shape[0]),oneprfY)
            star_psfx[istar]=sigX
            star_psfy[istar]=sigY
        
    all_sigx=star_psfx[np.logical_not(np.isnan(star_psfx))]
    all_sigy=star_psfy[np.logical_not(np.isnan(star_psfy))]
    all_sigx=all_sigx[all_sigx>2.4]
    all_sigy=all_sigy[all_sigy>2.4]
    print 'average prf(x) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigx),all_sigx.std(),np.median(all_sigx)*0.4*2.36,all_sigx.std()*0.4*2.26)
    print 'average prf(y) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigy),all_sigy.std(),np.median(all_sigy)*0.4*2.36,all_sigy.std()*0.4*2.26)
    image_psf[index,2]=np.median(all_sigx)
    image_psf[index,3]=np.median(all_sigy)
    image_psf[index,4]=all_sigx.std()
    image_psf[index,5]=all_sigy.std()

image 0 ==> NBSTARS = 1
average prf(x) = 4.47 +/- 0.00 pixels ==> psf  4.22 +/- 0.00 arcsec 
average prf(y) = 4.33 +/- 0.00 pixels ==> psf  4.08 +/- 0.00 arcsec 
image 1 ==> NBSTARS = 5
average prf(x) = 4.24 +/- 0.39 pixels ==> psf  4.00 +/- 0.36 arcsec 
average prf(y) = 4.09 +/- 0.18 pixels ==> psf  3.86 +/- 0.17 arcsec 


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


image 2 ==> NBSTARS = 21
average prf(x) = 3.76 +/- 0.24 pixels ==> psf  3.55 +/- 0.21 arcsec 
average prf(y) = 3.81 +/- 0.77 pixels ==> psf  3.60 +/- 0.69 arcsec 
image 3 ==> NBSTARS = 21
average prf(x) = 3.57 +/- 0.18 pixels ==> psf  3.37 +/- 0.16 arcsec 
average prf(y) = 3.96 +/- 0.65 pixels ==> psf  3.74 +/- 0.59 arcsec 
image 4 ==> NBSTARS = 10
average prf(x) = 3.55 +/- 0.39 pixels ==> psf  3.35 +/- 0.36 arcsec 
average prf(y) = 3.54 +/- 0.77 pixels ==> psf  3.34 +/- 0.70 arcsec 
image 5 ==> NBSTARS = 23
average prf(x) = 3.56 +/- 0.20 pixels ==> psf  3.36 +/- 0.18 arcsec 
average prf(y) = 3.50 +/- 0.78 pixels ==> psf  3.30 +/- 0.70 arcsec 
image 6 ==> NBSTARS = 33
average prf(x) = 3.41 +/- 0.21 pixels ==> psf  3.22 +/- 0.19 arcsec 
average prf(y) = 3.42 +/- 0.74 pixels ==> psf  3.23 +/- 0.67 arcsec 
image 7 ==> NBSTARS = 15
average prf(x) = 3.31 +/- 0.29 pixels ==> psf  3.13 +/- 0.26 arcsec 
average prf(y) = 3.41 +/- 0.32 pixels ==> psf  3.22 +/- 0.28 arcsec 
image 8 ==> NBSTARS = 

/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/numpy/core/_methods.py:94: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/numpy/core/_methods.py:116: RuntimeWarning:

image 17 ==> NBSTARS = 2
average prf(x) = nan +/- nan pixels ==> psf  nan +/- nan arcsec 
average prf(y) = 3.02 +/- 0.00 pixels ==> psf  2.85 +/- 0.00 arcsec 
image 18 ==> NBSTARS = 4
average prf(x) = nan +/- nan pixels ==> psf  nan +/- nan arcsec 
average prf(y) = 3.05 +/- 0.00 pixels ==> psf  2.88 +/- 0.00 arcsec 
image 19 ==> NBSTARS = 1
average prf(x) = nan +/- nan pixels ==> psf  nan +/- nan arcsec 
average prf(y) = 3.31 +/- 0.00 pixels ==> psf  3.13 +/- 0.00 arcsec 
image 20 ==> NBSTARS = 2
average prf(x) = nan +/- nan pixels ==> psf  nan +/- nan arcsec 
average prf(y) = nan +/- nan pixels ==> psf  nan +/- nan arcsec 
image 21 ==> NBSTARS = 44
average prf(x) = 3.29 +/- 0.25 pixels ==> psf  3.10 +/- 0.23 arcsec 
average prf(y) = 3.60 +/- 0.68 pixels ==> psf  3.39 +/- 0.61 arcsec 
image 22 ==> NBSTARS = 42
average prf(x) = 3.26 +/- 0.24 pixels ==> psf  3.08 +/- 0.22 arcsec 
average prf(y) = 3.42 +/- 0.79 pixels ==> psf  3.23 +/- 0.71 arcsec 
image 23 ==> NBSTARS = 49
average prf(x)

## Create astropy table

In [27]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [28]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,1,4.47415632464,4.32646900741,0.0,0.0
1,5,4.23915877347,4.09248732084,0.392718231946,0.18464392513
2,21,3.76020655028,3.81432116694,0.235852244067,0.767497075115
3,21,3.56680321661,3.96483104307,0.181167115677,0.648766625551
4,10,3.54780685321,3.54182907249,0.394432864272,0.769873402452
5,23,3.5644327671,3.49984527018,0.199997271512,0.77663355496
6,33,3.40986449559,3.42051094206,0.21440035444,0.74200994629
7,15,3.31082909982,3.41278053444,0.287048747871,0.315149038754
8,20,3.66129988733,3.56658136927,0.372296084714,0.601672154467


In [29]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [30]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [31]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str29,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD158485_grat_1.fits,2016-05-12T04:08:27.322,5.0,1,4.47415632464,4.32646900741,0.0,0.0
1,AssScImHD158485_grat_2.fits,2016-05-12T04:10:10.602,5.0,5,4.23915877347,4.09248732084,0.392718231946,0.18464392513
2,AssScImHD158485_grat_3.fits,2016-05-12T04:10:20.458,5.0,21,3.76020655028,3.81432116694,0.235852244067,0.767497075115
3,AssScImHD158485_grat_4.fits,2016-05-12T04:10:29.432,5.0,21,3.56680321661,3.96483104307,0.181167115677,0.648766625551
4,AssScImHD158485_grat_5.fits,2016-05-12T04:10:38.586,5.0,10,3.54780685321,3.54182907249,0.394432864272,0.769873402452
5,AssScImHD158485_grat_6.fits,2016-05-12T04:10:47.541,5.0,23,3.5644327671,3.49984527018,0.199997271512,0.77663355496
6,AssScImHD158485_grat_7.fits,2016-05-12T04:10:56.526,5.0,33,3.40986449559,3.42051094206,0.21440035444,0.74200994629
7,AssScImHD158485_grat_8.fits,2016-05-12T04:11:05.546,5.0,15,3.31082909982,3.41278053444,0.287048747871,0.315149038754
8,AssScImHD158485_grat_9.fits,2016-05-12T04:11:14.644,5.0,20,3.66129988733,3.56658136927,0.372296084714,0.601672154467


## Write output file

In [32]:
t.write(outputtablefile,format='fits')

## Pandas

In [33]:
df=t.to_pandas()

In [34]:
df.describe()

/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,101.000000,101.000000,101.000000,96.000000,99.000000,96.000000,99.000000
mean,50.000000,2.900990,22.534653,3.171685,3.549926,0.251827,0.419342
std,29.300171,2.140584,16.957927,0.317771,0.234563,0.176101,0.300821
min,0.000000,0.400000,1.000000,2.704380,3.019007,0.000000,0.000000
25%,25.000000,1.000000,8.000000,NaN,NaN,NaN,NaN
50%,50.000000,5.000000,17.000000,NaN,NaN,NaN,NaN
75%,75.000000,5.000000,41.000000,NaN,NaN,NaN,NaN
max,100.000000,5.000000,69.000000,4.474156,4.623810,0.998030,0.912355
